In [17]:
import boto3
from botocore.exceptions import ClientError
import json
import decimal

                                        Start session

In [11]:
dynamodb = boto3.resource('dynamodb',region_name='ap-south-1')



                                         Create table

In [5]:
try:
        table = dynamodb.create_table(
            TableName='Users',
            KeySchema=[
                {
                    'AttributeName': 'user_id',
                    'KeyType': 'HASH'  # Partition key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'user_id',
                    'AttributeType': 'S'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )
        table.wait_until_exists()
        print(f'Table status: {table.table_status}')
except ClientError as e:
        print(f"Error creating table: {e}")

Error creating table: An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: Users


                                       Insert the Data into Table

In [ ]:
try:
        response = table.put_item(
            Item=
            {
                'user_id': '00',
                'name': 'raju kumar',
                'age': 25,
                'email': 'sanjay@1245.com'
            }
        )
        print(f'PutItem succeeded: {response}')
except ClientError as e:
        print(f"Error inserting item: {e}")

                                         Read the table 

In [21]:
try:
        response = table.get_item(
            Key={
                'user_id': '003'
            }
        )
        item = response.get('Item')
        if item:
            print(f'GetItem succeeded: {item}')
        else:
            print('Item not found')
except ClientError as e:
        print(f"Error getting item: {e}")

GetItem succeeded: {'user_id': '003', 'email': 'sanjay.kumar@gamil.com', 'name': 'raju kumar', 'age': Decimal('89')}


                                                Update the table

In [25]:
try:
        response = table.update_item(
            Key={
                'user_id': '001'
            },
            UpdateExpression='SET age = :val1, email = :val2',
            ExpressionAttributeValues={
                ':val1': 56,
                ':val2': 'sanjay.kumar@gamil.com'
            },
            ReturnValues="ALL_NEW"
        )
        print(f'UpdateItem succeeded: {response}')

except ClientError as e:
        print(f"Error updating item: {e}")

UpdateItem succeeded: {'Attributes': {'user_id': '001', 'email': 'sanjay.kumar@gamil.com', 'name': 'John Doe', 'age': Decimal('56')}, 'ResponseMetadata': {'RequestId': 'HJBLTL9N26N76A4PUNMOFV9R8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 20 Sep 2024 10:26:09 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '118', 'connection': 'keep-alive', 'x-amzn-requestid': 'HJBLTL9N26N76A4PUNMOFV9R8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3362547469'}, 'RetryAttempts': 0}}


                                             Delete the Table

In [22]:
try:
        response = table.delete_item(
            Key={
                'user_id': '003'
            }
        )
        print(f'DeleteItem succeeded: {response}')

except ClientError as e:
        print(f"Error deleting item: {e}")

DeleteItem succeeded: {'ResponseMetadata': {'RequestId': 'U41NGJUVV8BFEDPP1MN74T3FI3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 20 Sep 2024 09:28:18 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'U41NGJUVV8BFEDPP1MN74T3FI3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


                                             Get item Count

In [4]:
print(table.item_count)

NameError: name 'table' is not defined

                                        Import Files into DynamoDB

In [18]:
dynamodb = boto3.resource('dynamodb')

In [11]:
table = dynamodb.Table('Users')

# Read the JSON data
with open('C:/Users/sanju/bridgelabpython/aws_repo/dynamodb/sample.json') as json_file:
    data = json.load(json_file)


# Insert each item into DynamoDB table
for item in data:
    table.put_item(Item=item)

print("Data imported successfully from dynamodb_data.json")


Data imported successfully from dynamodb_data.json


                                            Export the data table to file

In [19]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, decimal.Decimal):
            # Convert to int if the value is an integer, otherwise float
            return int(obj) if obj % 1 == 0 else float(obj)
        return super(DecimalEncoder, self).default(obj)

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('Users')

# Scan the table
response = table.scan()
data = response['Items']

# Write data to a JSON file using the custom encoder
with open('dynamodb_data.json', 'w') as json_file:
    json.dump(data, json_file, indent=4, cls=DecimalEncoder)

print("Data exported successfully to dynamodb_data.json")

Data exported successfully to dynamodb_data.json
